In [28]:
# Imports necesarios
import pandas as pd
import requests
import numpy as np
import time

try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
    
# Funcion para obtener el HTML de un link
def get_html(url):
    r = requests.get(url)
    html = BeautifulSoup(r.text)
    return html.body

def find_error_in_html(html, key, value):
    if key in ['Ended', 'Listing Ended']:
        return html.find(id=value)
    elif key in ['Sold out', 'Found Similar', 'Error']:
        return html.find(class_=value)
    else : 
        return None
    
    

# Variables necesarias
filename = "jordan_review_for_e_rav.xlsx"
filas_a_procesar = 8000 #Cambiar a -1 para procesar todas

# Ids a buscar, vienen del HTML
search_product_url = { 'Ended': 'w1-5-_msg',
                       #'Listing Ended': 'w1-6-_msg',
                       'Sold Out': 'outofstock',
                       'Found Similar': 'app-cvip-replacement-message',
                       'Error': 'error-header__headline',
                     }

# Leo el archivo en el dataframe
df = pd.read_excel("../spreadsheets/" + filename)

if filas_a_procesar != -1:
    df = df.head(filas_a_procesar)

# Saco las columnas que no voy a usar
cols_to_drop = ['site_id', 'MPN (ignore)']
df = df.drop(cols_to_drop, axis="columns")

#Agrego columnas nuevas para los datos procesados
df["YES/NO Procesado"] = np.nan
for key in search_product_url.keys():
    df[key] = np.nan

# Agrego el https:// a todos los links
df['product_line_url'] = df['product_line_url'].apply(lambda s: 'https://'+s)
df['product_url'] = df['product_url'].apply(lambda s: 'https://'+s)
df

,Product_line_id,Product_line_description,product_id,product_title (by seller),product_line_url,product_url,YES/NO,YES/NO Procesado,Error,Ended,Sold Out,Found Similar
0,7117656342,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",264737435029,NIKE AIR JORDAN 1 RETRO + NEUTRAL GREY 2001 SI...,https://www.ebay.com/b/bn_7117656342,https://www.ebay.com/itm/264737435029,y,NaN,NaN,NaN,NaN,NaN
1,7117717160,"[Brand = Jordan, Model = Jordan 'Why Not?' ZER...",164677534195,NEW Nike Jordan Why Not? Zer0.4 FACETASM Unise...,https://www.ebay.com/b/bn_7117717160,https://www.ebay.com/itm/164677534195,y,NaN,NaN,NaN,NaN,NaN
2,7117656574,"[Brand = Jordan, Model = Air Jordan 13, Silhou...",293808623286,Nike Air Jordan 2010 Retro 13 Varsity White Re...,https://www.ebay.com/b/bn_7117656574,https://www.ebay.com/itm/293808623286,y,NaN,NaN,NaN,NaN,NaN
3,7117651224,"[Brand = Jordan, Model = Air Jordan 5, Silhoue...",114780501812,2021 Nike Air Jordan Retro 5 Low Chinese New Y...,https://www.ebay.com/b/bn_7117651224,https://www.ebay.com/itm/114780501812,y,NaN,NaN,NaN,NaN,NaN
4,7117643400,"[Brand = Jordan, Model = Air Jordan 10, Silhou...",334065851300,Size 10 - Jordan 10 Retro NYC 2016,https://www.ebay.com/b/bn_7117643400,https://www.ebay.com/itm/334065851300,y,NaN,NaN,NaN,NaN,NaN
5,7117647055,"[Brand = Jordan, Model = Air Jordan 2, Silhoue...",114883811619,1994-1995 Nike Air Jordan II 2 Low Retro Chica...,https://www.ebay.com/b/bn_7117647055,https://www.ebay.com/itm/114883811619,y,NaN,NaN,NaN,NaN,NaN
6,7117649456,"[Brand = Jordan, Model = Jordan Flight Remix, ...",184714584239,Air Jordan Flight Remix Wolf Grey/Infrared 23-...,https://www.ebay.com/b/bn_7117649456,https://www.ebay.com/itm/184714584239,y,NaN,NaN,NaN,NaN,NaN
7,7117657277,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",334002458984,Jordan 1 Low TD Shadow 2019 Men’s Size 10 av52...,https://www.ebay.com/b/bn_7117657277,https://www.ebay.com/itm/334002458984,y,NaN,NaN,NaN,NaN,NaN
8,7117648427,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",193887868653,Size 9 - Jordan 1 High Zoom Air Comfort Red,https://www.ebay.com/b/bn_7117648427,https://www.ebay.com/itm/193887868653,y,NaN,NaN,NaN,NaN,NaN
9,7117648509,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",333958246029,Size 8.5 - Jordan 1 Retro High OG Carmine DS S...,https://www.ebay.com/b/bn_7117648509,https://www.ebay.com/itm/333958246029,y,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Voy a recorrer todas las filas buscandos los ids
# Para cada id voy a agregar una columna a la tabla mas una columna con el YES/NO total
for ind, row in df.iterrows():
    print("Procesando fila %d"%(ind+1), end='\r')
    
    #Cada 1000 consultas espero un minuto
    if ind > 0 and ind%1000 == 0:
        time.sleep(60)
        
        
    # Obtengo la URL (product_url)
    retries = 0
    while(retries < 5):
        try:  
            html = get_html(row['product_url'])
            break
        except:
            print("Fallo la conexion intentando nuevamente en 30 segundos")
            time.sleep(30)
            retries = retries+1
    
    # Busco los ids en la url
    for index, (key, value) in enumerate(search_product_url.items()):
        match = find_error_in_html(html, key, value)
        if match is not None:
            #print(match)
            df.loc[ind, key] = "SI" 
            df.loc[ind,"YES/NO Procesado"] = "n"
            break
            
# Guardo los resultado en una nueva planilla
result_filename = filename.replace('spreadsheets', 'results').split('.')[0] + '_processed.xlsx'
df.to_excel('../results/'+result_filename)
    
df

<div class="app-cvip-replacement-message app-cvip-replacement-message-product">Encontramos un artículo parecido</div>
<span class="msgTextAlign" id="w1-5-_msg">Este anuncio finalizó. </span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor final

<div class="app-cvip-replacement-message app-cvip-replacement-message-product">Encontramos un artículo parecido</div>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<div class="app-cvip-replacement-message app-cvip-replacement-message-product">Encontramos un artículo parecido</div>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<div class="app-cvip-replacement-message app-cvip-replacement-message-product">Encontramos un artículo parecido</div>
<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>
<div class="app-cvip-

In [3]:
# Imprimo algunas estadisticas
procesadas = len(df.index) 
completadas = len(df.loc[~pd.isna(df["YES/NO Procesado"])].index)

print("Filas procesadas: {}. Filas completadas: {}".format(procesadas, completadas))

Filas procesadas: 50. Filas completadas: 10


In [4]:
#Chequeo los datos. Comparamos la columna YES/NO con el resultado del proceso. Por ahora se completan solo los no
check_df = df.loc[~pd.isna(df['YES/NO'])]
comp = np.where((check_df["YES/NO Procesado"] == "n") & (check_df["YES/NO"] == "n") | (pd.isna(check_df["YES/NO Procesado"])), True, False)

if not all(comp):
    print("Las siguientes filas no coinciden con lo procesado:")
    print(list(np.where(comp == False)[0]+2))
else:
    print("Todas las filas coinciden!")


Todas las filas coinciden!


<span class="msgTextAlign" id="w1-5-_msg">El vendedor finalizó este anuncio porque el artículo ya no está disponible.</span>